# Setting up PostgreSQL and pgAdmin4 on Ubuntu

## Install the correct packages

I started by looking at [PostgreSQL's download page](https://www.postgresql.org/download/linux/ubuntu/). You can choose to add their repositories to your apt repository list, or I took their hint about using the packages already available in the Ubuntu distribution.

The following packages are installed on my system now:

```
postgresql-10
postgresql-client-10
postgresql-client-common
postgresql-common
postgresql-contrib
postgresql-server-dev-10

libpq-dev
libpq5

build-essential
libssl-dev
libffi-dev
libgmp3-dev
virtualenv

pgadmin4-apache2
```

Some of these are dependent on each other, so you probably don't have to list all of them when you run 'sudo apt install ...'. But it won't hurt.

The package pgadmin4-apache2 may ask about an initial user to configure, with a default value of "postgres@localhost". One of the postgres packages creates this account on your machine, so this default value is fine.

I may also have missed some dependencies, so if apt indicates it wants to install additional packages, let it do that.

## Download and install pgadmin4 via a Python wheel

PostgreSQL's download page, and some of the pages I link to below, suggest that pgadmin4 can be installed via apt as well, but I was not able to find this package.

What I found after some searching is [the pgadmin4 download page](https://www.pgadmin.org/download/pgadmin-4-python-wheel/), from which you can download the latest Python wheel.

I leaned heavily on [these instructions by Nikhil Krishnan](https://medium.com/@nikhilkrishnan/how-to-install-pgadmin4-in-ubuntu-16-04-with-python3-aecc79de4b7d). 

Click on the latest version on the download page (for me, it was v4.3, released 2019/03/07).

You'll be taken to a page with a link to a .whl file (pgadmin4-4.3-py2.py3-none-any.whl, for me). Copy the URL for this .whl file, then use wget to retrieve it from the command line.

`wget <url>`

The .whl file will end up in your current working directory. Make sure you know which directory this is--it will be needed later.

## Create a virtual environment in which pgAdmin4 will run

Continuing from the instructions in the section above--create a directory that can serve as the home for a small virtual environment, and install the Python wheel file's contents into this virtual environment. PgAdmin4 will run from this directory, in this virtual environment.

I'm cribbing a lot of the commands in this section from Nikhil Krishnan's instructions...with comments where needed. :)

The suggested location is ~/py3-venv-pgadmin, and that works.

`mkdir ~/py3-venv-pgadmin`

Many of the instructions in this section are dependent on which version of Python 3 you are running. Nikhil's instructions are for 3.5 specficially. I ran this command to see what was running on my machine:

```
$ python3 --version
Python 3.6.7
```

Just 3.6 (or whatever major + minor version you have) should be enough to complete the following instructions.

Create the virtual environment, replacing "<version>" with the version of Python 3 you're running:

`virtualenv --system-site-packages --no-setuptools --python=python<version> ~/py3-venv-pgadmin`

Change into the bin directory in the virtual environment you created, then activate the virtualenv:

`cd ~/py3-venv-pgadmin/bin`

`source activate`

Your prompt will change if everything is working--mine was prefixed with the name of the top-level virtual directory in parenthesis:

```
(py3-venv-pgadmin)
```

You can verify pip is working in this virtual environment by simply invoking it:

`~/py3-venv-pgadmin/bin/pip3`

It will return the Usage text if it's set up correctly.

Now--__from the prompt that indicates you are in the virtual environment__--install pgAdmin4 via the Python wheel you downloaded:

`pip3 install pgadmin4–2.0-py2.py3-none-any.whl`

If you incorrectly attempt to install this .whl file from _outside_ of the virtual environment, you'll get weird permissions errors.

Once the contents of the .whl file have been successfully installed, you need to create a config file for pgadmin4:

`cd ~/py3-venv-pgadmin/lib/python3.5/site-packages/pgadmin4`

`touch config_local.py`

`nano config_local.py` <--or vi, or emacs, or whatever. Point is, edit this file :)

If you copy/paste Nikhil's config file, you're going to end up switching all of the quotes from "smart" paired quotes to single quotes that can be read by pgadmin4.

Copy/paste this into your copy of config_local.py instead:

```
import os
SERVER_MODE = False
DATA_DIR = os.path.realpath(os.path.expanduser(u'~/.pgadmin/'))
LOG_FILE = os.path.join(DATA_DIR, 'pgadmin4.log')
SQLITE_PATH = os.path.join(DATA_DIR, 'pgadmin4.db')
SESSION_DB_PATH = os.path.join(DATA_DIR, 'sessions')
STORAGE_DIR = os.path.join(DATA_DIR, 'storage')
```

With this config file in place, you can set up the pgAdmin4 environment:

`python3 ~/py3-venv-pgadmin/lib/python3.5/site-packages/pgadmin4/setup.py`

If all went well to this point, you're ready to try running pgAdmin4! 

`python3 ~/py3-venv-pgadmin/lib/python3.5/site-packages/pgadmin4/pgAdmin4.py`

It will log directly to your term window, and one of the last things it will print is a URL, like: http://127.0.0.1:5050/

Copy/paste the URL it prints into a fresh tab of your web browser. You should see the main dashboard of pgAdmin4. Congrats!

Go ahead and close the browser tab, and break out of the running script in your term window (Ctrl-C or its equivalent).

Every time you want to run pgAdmin4 now, you will need to cd to the venv directory, start the virtual environment, and call the pgAdmin4 Python script, as you just did. Writing a script to do this work for you will make things easier.

I'm not sure what to make of Nikhil's script--I wrote my own, which simply does the three things I just mentioned:

`nano ~/py3-venv-pgadmin/pgadmin4.sh`

```
#!/bin/bash

cd ~/py3-venv-pgadmin/bin
source activate
python3 ~/py3-venv-pgadmin/lib/python3.6/site-packages/pgadmin4/pgAdmin4.py

```

Setting up an alias to call this script by simply entering `pgadmin4` at a prompt, as Nikhil suggests, is a great idea. 

My account was set up with a separate ~/.bash_aliases file, that is sourced by ~/.bashrc. You can add the following line to the end of either file, again using your editor of choice:

```
alias pgadmin4='~/py3-venv-pgadmin/pgadmin4.sh'
```



## Configuring the postgresql userid

[This page](https://websiteforstudents.com/install-postgresql-server-11-and-manage-it-with-pgadmin4-on-ubuntu-16-04-18-04-18-10/) has some decent instructions to configure the postgresql userid, so you can actually connect to a database once you have pgAdmin4 running. I missed this step the first time. :)


[I also used some instructions from this page](https://www.digitalocean.com/community/tutorials/how-to-install-and-use-postgresql-on-ubuntu-18-04) to create a PostgreSQL id for myself. I'm not sure if it matters that you are connecting as the postgresql userid, or your own, so consider this section optional.

